In [915]:
import requests
from urllib import parse
import pprint
pp = pprint.PrettyPrinter(indent=4)
import pandas as pd
import json
import numpy as np
import sys
from copy import deepcopy

In [916]:
pd.set_option('display.max_rows', None)

# 챔피언, 특성(시너지), 아이템 json 파일 불러오기

In [917]:
with open("ko_kr.json", 'r') as file:
    data = json.load(file)

## 챔피언 이름 영어 -> 한국어

In [918]:
def name_eng_kr(data_champions) :
    champ_dict = dict()
    for i in range(data_champions.shape[0]) :
        champ_name = data_champions.apiName[i]
        champ_name_kr = data_champions.name[i]
        champ_dict[champ_name] = champ_name_kr
    return champ_dict

In [919]:
list_champions = pd.DataFrame(data['setData']).loc[0,:]['champions'].copy()
data_champions = pd.DataFrame(list_champions)
champ_kr = name_eng_kr(data_champions)
champ_kr

{'TFT7_Vladimir': '블라디미르',
 'TFT7_Nidalee': '니달리',
 'TFT7_Senna': '세나',
 'TFT7_Sett': '세트',
 'TFT7_Gnar': '나르',
 'TFT7_Sejuani': '세주아니',
 'TFT7_Ezreal': '이즈리얼',
 'TFT7_Skarner': '스카너',
 'TFT7_Yone': '요네',
 'TFT7_Varus': '바루스',
 'TFT7_LeeSin': '리 신',
 'TFT7_Shyvana': '쉬바나',
 'TFT7_Hecarim': '헤카림',
 'TFT7_Soraka': '소라카',
 'TFT7_Karma': '카르마',
 'TFT7_Zoe': '조이',
 'TFT7_Yasuo': '야스오',
 'TFT7_Nunu': '누누와 윌럼프',
 'TFT7_Bard': '바드',
 'TFT7_IvernMinion': '조이의 데이지',
 'TFT7_Xayah': '자야',
 'TFT7_Qiyana': '키아나',
 'TFT7_Sylas': '사일러스',
 'TFT7_Leona': '레오나',
 'TFT7_Twitch': '트위치',
 'TFT6_TheGoldenEgg': '황금 알',
 'TFT7_AoShin': '아오 신',
 'TFT7_DragonPurple': '사이펜',
 'TFT7_DragonGreen': '시오유',
 'TFT7_DragonGold': '이다스',
 'TFT7_DragonBlue': '다에야',
 'TFT7_JadeStatue': '비취 조각상',
 'TFT7_AurelionSol': '아우렐리온 솔',
 'TFT7_AquaticDragon': '솜',
 'TFT7_Taliyah': '탈리야',
 'TFT7_Malphite': '말파이트',
 'TFT7_Zac': '자크',
 'TFT7_Aphelios': '아펠리오스',
 'TFT7_Rell': '렐',
 'TFT7_Zyra': '자이라',
 'TFT7_Kaisa': '카이사',
 'TFT7_Seraphi

In [909]:
data_champions

,ability,apiName,cost,icon,name,stats,traits
0,{'desc': '블라디미르가 대상에게 @ModifiedDamage@의 마법 피해를...,TFT7_Vladimir,1,ASSETS/UX/TFT/ChampionSplashes/TFT7_Vladimir.T...,블라디미르,"{'armor': 45.0, 'attackSpeed': 0.6000000238418...","[별, 요술사]"
1,{'desc': '니달리가 전투가 끝날 때까지 쿠거로 변신해 공격 사거리가 1로 줄...,TFT7_Nidalee,1,ASSETS/UX/TFT/ChampionSplashes/TFT7_Nidalee.TF...,니달리,"{'armor': 30.0, 'attackSpeed': 0.6999999880790...","[별, 형상변환자]"
2,{'desc': '세나가 가장 멀리 떨어진 적에게 검은 안개를 뿌려 처음 적중한 적...,TFT7_Senna,1,ASSETS/UX/TFT/ChampionSplashes/TFT7_Senna.TFT_...,세나,"{'armor': 15.0, 'attackSpeed': 0.6999999880790...","[분노날개, 포병대]"
3,{'desc': '다음 @Duration@초 동안 세트가 방어력과 마법 저항력을 @...,TFT7_Sett,1,ASSETS/UX/TFT/ChampionSplashes/TFT7_Sett.TFT_S...,세트,"{'armor': 40.0, 'attackSpeed': 0.6999999880790...","[분노날개, 용술사]"
4,{'desc': '나르가 메가 나르로 변신해 대상에게 뛰어들고 근처 적을 @Stun...,TFT7_Gnar,2,ASSETS/UX/TFT/ChampionSplashes/TFT7_Gnar.TFT_S...,나르,"{'armor': 40.0, 'attackSpeed': 0.6999999880790...","[비취, 형상변환자]"
5,{'desc': '세주아니가 철퇴를 넓게 휘둘러 원뿔 범위의 모든 적을 공격하고 <...,TFT7_Sejuani,1,ASSETS/UX/TFT/ChampionSplashes/TFT7_Sejuani.TF...,세주아니,"{'armor': 40.0, 'attackSpeed': 0.5500000119209...","[길드, 기병대]"
6,{'desc': '이즈리얼이 대상에게 에너지 광선을 발사합니다. 처음 적중당한 적은...,TFT7_Ezreal,1,ASSETS/UX/TFT/ChampionSplashes/TFT7_Ezreal.TFT...,이즈리얼,"{'armor': 15.0, 'attackSpeed': 0.6999999880790...","[폭풍, 민첩사수]"
7,{'desc': '스카너가 @Duration@초 동안 @ModifiedShield@...,TFT7_Skarner,1,ASSETS/UX/TFT/ChampionSplashes/TFT7_Skarner.TF...,스카너,"{'armor': 45.0, 'attackSpeed': 0.6000000238418...","[별, 난동꾼]"
8,{'desc': '기본 지속 효과: 요네의 공격이 @ModifiedBonusDama...,TFT7_Yone,2,ASSETS/UX/TFT/ChampionSplashes/TFT7_Yone.TFT_S...,요네,"{'armor': 40.0, 'attackSpeed': 0.8500000238418...","[신기루, 전사]"
9,{'desc': '바루스가 우주의 덩굴을 대상에게 보내 처음 적중한 적에게 <sca...,TFT7_Varus,3,ASSETS/UX/TFT/ChampionSplashes/TFT7_Varus.TFT_...,바루스,"{'armor': 25.0, 'attackSpeed': 0.6999999880790...","[별, 민첩사수]"


# traits(특성) json 파일 불러오기

In [922]:
data_traits = pd.DataFrame(data['setData']).loc[0,:]['traits']
traits = pd.DataFrame(data_traits)
traits

,apiName,desc,effects,icon,name
0,Set7_Cannoneer,5번째 공격마다 대포를 발사하고 대상 주변에 폭발을 일으켜 물리 피해를 입힙니다.<...,"[{'maxUnits': 3, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_5_Cannoneer.tex,포병대
1,Set7_Tempest,@DelayTime@초 후에 전장에 벼락이 내립니다. 적이 @StunDuration...,"[{'maxUnits': 3, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_7_Tempest.tex,폭풍
2,Set7_Assassin,고유: 전투 시작 시 적의 후방으로 도약합니다.<br><br>암살자는 추가 치명타 ...,"[{'maxUnits': 3, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_6_Assassin.tex,암살자
3,Set7_Prodigy,(@MinUnits@) 뇸뇸이는 게임마다 무작위 특성을 얻습니다. 이전 조련사를 소...,"[{'maxUnits': 25000, 'minUnits': 3, 'style': 3...",ASSETS/UX/TraitIcons/Trait_Icon_Prodigy.TFT_Se...,신동
4,Set7_Dragon,고유: 2개의 팀 슬롯이 필요합니다. <br><br>용은 특성 중첩을 3 증가시키고...,"[{'maxUnits': 1, 'minUnits': 1, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_7_Dragon.tex,용
5,Set7_Ragewing,고유: 마나를 분노로 전환합니다. 기본 공격 시 분노를 15 생성합니다. 스킬을 사...,"[{'maxUnits': 3, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_7_Ragewing.tex,분노날개
6,Set7_Guild,아군에게 고유 보너스를 부여합니다. 길드원은 두 배로 얻습니다. 전장의 길드 아군마...,"[{'maxUnits': 1, 'minUnits': 1, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_7_Guild.tex,길드
7,Set7_Cavalier,고유: 이동 시 대상에게 빠르게 돌격합니다.<br><br>기병대가 방어력과 마법 저...,"[{'maxUnits': 2, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_5_Cavalry.tex,기병대
8,Set7_Mage,요술사는 스킬을 두 번 사용하고 총 주문력이 조정됩니다.<br><br><expand...,"[{'maxUnits': 4, 'minUnits': 3, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_2_Mage.tex,요술사
9,Set7_Evoker,아군 또는 적이 스킬을 사용할 때마다 일깨우는 자가 마나를 얻습니다.<br><br>...,"[{'maxUnits': 2, 'minUnits': 2, 'style': 1, 'v...",ASSETS/UX/TraitIcons/Trait_Icon_7_Evoker.tex,일깨우는 자


# item 조합표 만들기

In [932]:
items = pd.DataFrame(data['items'])

## item raw data 에서 item 만 추출

In [933]:
def make_item_list(items) :
    n = len(items)
    basic = set([i for i in range(1,10)])
    data = pd.DataFrame(columns = items.columns)
    for i in range(n) :
        if items['id'][i] in basic :
            data = pd.concat([data, items.iloc[i:(i+1), :]])
        else :
            item_from = set(items['from'][i])
            if len(item_from) != 0 and item_from.issubset(basic) :
                data = pd.concat([data, items.iloc[i:(i+1), :]])
            
    return data

items_comb = make_item_list(items).reset_index(drop = True)

In [943]:
items_comb

,apiName,desc,effects,from,icon,id,name,unique,season
0,TFT7_Item_MirageEmblemItem,장착 시 신기루 특성 획득<br><br><tftitemrules>[고유 - 중복 적...,{'MagicResist': 20.0},"[6, 8]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,2314,신기루 상징,True,7
1,TFT7_Item_SwiftshotEmblemItem,장착 시 민첩사수 특성 획득<br><br><tftitemrules>[고유 - 중복 ...,{'AS': 10.0},"[8, 2]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,2313,민첩사수 상징,True,7
2,TFT7_Item_CavalierEmblemItem,장착 시 기병대 특성 획득<br><br><tftitemrules>[고유 - 중복 적...,{'Armor': 20.0},"[8, 5]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,2307,기병대 상징,True,7
3,TFT7_Item_GuardianEmblemItem,장착 시 수호자 특성 획득<br><br><tftitemrules>[고유 - 중복 적...,{'Health': 150.0},"[8, 7]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,2323,수호자 상징,True,7
4,TFT7_Item_DragonmancerEmblemItem,장착 시 용술사 특성 획득,{'AP': 10.0},"[8, 3]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,2426,용술사 상징,True,7
5,TFT7_Item_MageEmblemItem,장착 시 요술사 특성 획득<br><br><tftitemrules>[고유 - 중복 적...,{'Mana': 15.0},"[8, 4]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,2311,요술사 상징,True,7
6,TFT7_Item_LagoonEmblemItem,장착 시 석호 특성 획득<br><br><tftitemrules>[고유 - 중복 적용...,"{'CritChance': 10.0, '{c4b5579c}': 5.0}","[9, 8]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,2100,석호 상징,True,7
7,TFT7_Item_ShimmerscaleEmblemItem,장착 시 빛비늘 특성 획득<br><br><tftitemrules>[고유 - 중복 적...,{'AD': 10.0},"[8, 1]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,2302,빛비늘 상징,True,7
8,TFT6_Item_ChallengerEmblemItem,장착 시 도전자 특성 획득<br><br><tftitemrules>[고유 - 중복 적...,{'AS': 10.0},"[2, 8]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,28,도전자 상징,True,6
9,TFT6_Item_MutantEmblemItem,장착 시 돌연변이 특성 획득<br><br><tftitemrules>[고유 - 중복 ...,{'MagicResist': 20.0},"[6, 8]",ASSETS/Maps/Particles/TFT/Item_Icons/Traits/Sp...,2190,돌연변이 상징,True,6


## 시즌 정보 추출

In [934]:
# 시즌 정보
items_comb['season'] = 0 * len(items_comb)
for i in range(len(items_comb)) :
    season = items_comb['apiName'][i].split('_')[0]
    if season == 'TFT' :
        season = 1
    else :
        season = season[3:]
    items_comb.loc[i, ['season']] = int(season)
items_comb = items_comb.sort_values(by = ['season'], ascending = False).reset_index(drop = True)

## 조합표 만들기

In [935]:
items_mat = [[0 for _ in range(10)] for _ in range(10)]
for i in range(len(items_comb)) :
    name = items_comb['name'][i]
    code = items_comb['id'][i]
    c = items_comb['from'][i]
    if not items_comb['from'][i] :
        items_mat[code][0] = name
        items_mat[0][code] = name
    else :
        if items_mat[c[0]][c[1]] == 0 or items_mat[c[1]][c[0]] == 0 :
            items_mat[c[0]][c[1]] = name
            items_mat[c[1]][c[0]] = name
items_mat = pd.DataFrame(items_mat)

In [936]:
items_mat

,0,1,2,3,4,5,6,7,8,9
0,0,B.F. 대검,곡궁,쓸데없이 큰 지팡이,여신의 눈물,쇠사슬 조끼,음전자 망토,거인의 허리띠,뒤집개,연습용 장갑
1,B.F. 대검,죽음의 검,거인 학살자,마법공학 총검,쇼진의 창,밤의 끝자락,피바라기,지크의 전령,빛비늘 상징,무한의 대검
2,곡궁,거인 학살자,고속 연사포,구인수의 격노검,스태틱의 단검,거인의 결의,루난의 허리케인,즈롯 차원문,민첩사수 상징,최후의 속삭임
3,쓸데없이 큰 지팡이,마법공학 총검,구인수의 격노검,라바돈의 죽음모자,대천사의 지팡이,강철의 솔라리 펜던트,이온 충격기,모렐로노미콘,용술사 상징,보석 건틀릿
4,여신의 눈물,쇼진의 창,스태틱의 단검,대천사의 지팡이,푸른 파수꾼,수호자의 맹세,힘의 성배,구원,요술사 상징,정의의 손길
5,쇠사슬 조끼,밤의 끝자락,거인의 결의,강철의 솔라리 펜던트,수호자의 맹세,덤불 조끼,가고일 돌갑옷,태양불꽃 망토,기병대 상징,침묵의 장막
6,음전자 망토,피바라기,루난의 허리케인,이온 충격기,힘의 성배,가고일 돌갑옷,용의 발톱,서풍,신기루 상징,수은
7,거인의 허리띠,지크의 전령,즈롯 차원문,모렐로노미콘,구원,태양불꽃 망토,서풍,워모그의 갑옷,수호자 상징,밴시의 발톱
8,뒤집개,빛비늘 상징,민첩사수 상징,용술사 상징,요술사 상징,기병대 상징,신기루 상징,수호자 상징,전략가의 왕관,석호 상징
9,연습용 장갑,무한의 대검,최후의 속삭임,보석 건틀릿,정의의 손길,침묵의 장막,수은,밴시의 발톱,석호 상징,도적의 장갑


# 증강체

In [976]:
def make_augment_list(augments) :
    data = pd.DataFrame(columns = items.columns)
    for i in range(len(augments)) :
        if 'Augment' in augments.loc[i,'apiName'] :
            data = pd.concat([data, pd.DataFrame(items.iloc[i,:]).T], axis = 0)
            
    return data

def aug_eng_kr(augments) :
    aug_dict = dict()
    for i in augments.index :
        aug_dict[augments.loc[i,'apiName']] = augments.loc[i,'name']
    return aug_dict

augments = make_augment_list(items)
aug_dict = aug_eng_kr(augments)

# load API key from txt file

In [883]:
def getAPIkey():
    f = open("./riot_api.txt","r")
    return f.read()

In [884]:
api_key = getAPIkey()

In [885]:
api_key

'RGAPI-d004f431-5058-4ef5-adf2-6b6d5b66b15d'

In [886]:
# request headers
request_header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

In [887]:
# response body
# {
#     "id": "GZaWqjWrS1qLD2HUON3lUkh9ap80UpTV7pK8bNMhLQv943U",
#     "accountId": "YewhyRy4Dg5VU_wCcdGpCZsd4kvH4uFpXK0xT1FfjYpALE4",
#     "puuid": "i_2Yr1cjIXc0UR_M_i8JZr_8WN5Uhq0NRQNWFlTIN_IDpEhk4-jYt13etJQjEnt3c-Ix925M6cYVBA",
#     "name": "예비역협회장",
#     "profileIconId": 5031,
#     "revisionDate": 1667979235000,
#     "summonerLevel": 97
# }

# 소환사 정보 구현하기

## 소환사명, puuid, accountid, id 통해서 소환사 검색

### 소환사명으로 검색

In [888]:
def summoner_v1_by_summoner_name(summonerName):
    encodingSummonerName = parse.quote(summonerName)
    url = f"https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/{encodingSummonerName}"
    return requests.get(url, headers=request_header).json()

In [889]:
pp.pprint(summoner_v1_by_summoner_name("예비역협회장"))

{   'accountId': 'YewhyRy4Dg5VU_wCcdGpCZsd4kvH4uFpXK0xT1FfjYpALE4',
    'id': 'GZaWqjWrS1qLD2HUON3lUkh9ap80UpTV7pK8bNMhLQv943U',
    'name': '예비역협회장',
    'profileIconId': 5031,
    'puuid': 'i_2Yr1cjIXc0UR_M_i8JZr_8WN5Uhq0NRQNWFlTIN_IDpEhk4-jYt13etJQjEnt3c-Ix925M6cYVBA',
    'revisionDate': 1668579059000,
    'summonerLevel': 97}


### Account id 로 검색

In [890]:
def summoner_v1_by_accountId(accountId):
    url = f"https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-account/{accountId}"
    return requests.get(url, headers=request_header).json()

In [891]:
pp.pprint(summoner_v1_by_accountId("YewhyRy4Dg5VU_wCcdGpCZsd4kvH4uFpXK0xT1FfjYpALE4"))

{   'accountId': 'YewhyRy4Dg5VU_wCcdGpCZsd4kvH4uFpXK0xT1FfjYpALE4',
    'id': 'GZaWqjWrS1qLD2HUON3lUkh9ap80UpTV7pK8bNMhLQv943U',
    'name': '예비역협회장',
    'profileIconId': 5031,
    'puuid': 'i_2Yr1cjIXc0UR_M_i8JZr_8WN5Uhq0NRQNWFlTIN_IDpEhk4-jYt13etJQjEnt3c-Ix925M6cYVBA',
    'revisionDate': 1668579059000,
    'summonerLevel': 97}


### puuid 로 검색

In [892]:
def summoner_v1_by_puuid(puuid):
    url = f"https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-puuid/{puuid}"
    return requests.get(url, headers=request_header).json()

In [893]:
pp.pprint(summoner_v1_by_puuid('i_2Yr1cjIXc0UR_M_i8JZr_8WN5Uhq0NRQNWFlTIN_IDpEhk4-jYt13etJQjEnt3c-Ix925M6cYVBA'))

{   'accountId': 'YewhyRy4Dg5VU_wCcdGpCZsd4kvH4uFpXK0xT1FfjYpALE4',
    'id': 'GZaWqjWrS1qLD2HUON3lUkh9ap80UpTV7pK8bNMhLQv943U',
    'name': '예비역협회장',
    'profileIconId': 5031,
    'puuid': 'i_2Yr1cjIXc0UR_M_i8JZr_8WN5Uhq0NRQNWFlTIN_IDpEhk4-jYt13etJQjEnt3c-Ix925M6cYVBA',
    'revisionDate': 1668579059000,
    'summonerLevel': 97}


### id 로 검색

In [894]:
def summoner_v1_summoner_name(id):
    url = f"https://kr.api.riotgames.com/tft/summoner/v1/summoners/{id}"
    return requests.get(url, headers=request_header).json()

In [895]:
pp.pprint(summoner_v1_summoner_name("GZaWqjWrS1qLD2HUON3lUkh9ap80UpTV7pK8bNMhLQv943U"))

{   'accountId': 'YewhyRy4Dg5VU_wCcdGpCZsd4kvH4uFpXK0xT1FfjYpALE4',
    'id': 'GZaWqjWrS1qLD2HUON3lUkh9ap80UpTV7pK8bNMhLQv943U',
    'name': '예비역협회장',
    'profileIconId': 5031,
    'puuid': 'i_2Yr1cjIXc0UR_M_i8JZr_8WN5Uhq0NRQNWFlTIN_IDpEhk4-jYt13etJQjEnt3c-Ix925M6cYVBA',
    'revisionDate': 1668579059000,
    'summonerLevel': 97}


# puuid 에서 Match 로드

In [896]:
def match_v1_get_list_match_id(puuid):
    url = f"https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?start=0&count=20"
    return requests.get(url, headers=request_header).json()

In [897]:
match_v1_get_list_match_id('i_2Yr1cjIXc0UR_M_i8JZr_8WN5Uhq0NRQNWFlTIN_IDpEhk4-jYt13etJQjEnt3c-Ix925M6cYVBA')

['KR_6219216312',
 'KR_6216559852',
 'KR_6210966508',
 'KR_6208570199',
 'KR_6208370312',
 'KR_6203295104',
 'KR_6202775387',
 'KR_6198887107',
 'KR_6198247318',
 'KR_6198100416',
 'KR_6198056901',
 'KR_6196640803',
 'KR_6189441839',
 'KR_6189406274',
 'KR_6189332676',
 'KR_6188617511',
 'KR_6188615131',
 'KR_6188521062',
 'KR_6187434409',
 'KR_6186451822']

# Match data 불러오기

In [898]:
def load_match(matchid) :
    url = f"https://asia.api.riotgames.com/tft/match/v1/matches/{matchid}"
    return requests.get(url, headers=request_header).json()

In [899]:
match = load_match('KR_6208570199')

In [900]:
match.keys()

dict_keys(['metadata', 'info'])

In [999]:
match_metadata = match['metadata']
match_info = match['info']

match_metadata.keys()   # dict_keys(['data_version', 'match_id', 'participants'])
match_info.keys()
# dict_keys(['game_datetime', 'game_length', 'game_version', 'participants', 'queue_id',
#            'tft_game_type', 'tft_set_core_name', 'tft_set_number'])
match_info['participants'][0].keys()
# dict_keys(['augments', 'companion', 'gold_left', 'last_round', 'level', 'placement',
#            'players_eliminated', 'puuid', 'time_eliminated', 'total_damage_to_players', 'traits', 'units'])

dict_keys(['augments', 'companion', 'gold_left', 'last_round', 'level', 'placement', 'players_eliminated', 'puuid', 'time_eliminated', 'total_damage_to_players', 'traits', 'units'])

In [1004]:
match_result = pd.DataFrame(match_info['participants'])

In [1005]:
pd.DataFrame(match_result['units'][1])

,character_id,itemNames,items,name,rarity,tier
0,TFT7_Yone,[],[],,1,2
1,TFT7_Yone,[],[],,1,2
2,TFT7_Zyra,[TFT_Item_StatikkShiv],[24],,1,2
3,TFT7_Jax,[],[],,1,2
4,TFT7_Sylas,"[TFT_Item_ZekesHerald, TFT_Item_PowerGauntlet]","[17, 79]",,2,1
5,TFT7_Olaf,[],[],,2,2
6,TFT7_DragonPurple,"[TFT_Item_TitansResolve, TFT_Item_GuardianAngel]","[25, 94]",,5,2
7,TFT7_Pantheon,[TFT_Item_TitansResolve],[25],,4,1


In [907]:
match_result['traits'][0]

[{'name': 'Set7_Astral',
  'num_units': 8,
  'style': 3,
  'tier_current': 3,
  'tier_total': 3},
 {'name': 'Set7_Bruiser',
  'num_units': 2,
  'style': 1,
  'tier_current': 1,
  'tier_total': 4},
 {'name': 'Set7_Dragon',
  'num_units': 1,
  'style': 1,
  'tier_current': 1,
  'tier_total': 6},
 {'name': 'Set7_Evoker',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 3},
 {'name': 'Set7_Mage',
  'num_units': 5,
  'style': 2,
  'tier_current': 2,
  'tier_total': 4},
 {'name': 'Set7_Shapeshifter',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 2},
 {'name': 'Set7_Shimmerscale',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 4},
 {'name': 'Set7_SpellThief',
  'num_units': 1,
  'style': 3,
  'tier_current': 1,
  'tier_total': 1},
 {'name': 'Set7_Swiftshot',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 4},
 {'name': 'Set7_Whispers',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 3}]

In [994]:
match_result.units[0]

[{'character_id': 'TFT7_Vladimir',
  'itemNames': ['TFT_Item_ThiefsGloves',
   'TFT_Item_Chalice',
   'TFT_Item_PowerGauntlet'],
  'items': [99, 46, 79],
  'name': 'Vladimir',
  'rarity': 0,
  'tier': 3},
 {'character_id': 'TFT7_Skarner',
  'itemNames': ['TFT_Item_IonicSpark'],
  'items': [36],
  'name': 'Skarner',
  'rarity': 0,
  'tier': 3},
 {'character_id': 'TFT7_Nidalee',
  'itemNames': [],
  'items': [],
  'name': 'Nidalee',
  'rarity': 0,
  'tier': 3},
 {'character_id': 'TFT7_Lux',
  'itemNames': ['TFT_Item_ArchangelsStaff',
   'TFT_Item_JeweledGauntlet',
   'TFT_Item_InfinityEdge'],
  'items': [34, 39, 19],
  'name': '',
  'rarity': 1,
  'tier': 3},
 {'character_id': 'TFT7_Sylas',
  'itemNames': ['TFT_Item_BrambleVest',
   'TFT_Item_GargoyleStoneplate',
   'TFT4_Item_OrnnEternalWinter'],
  'items': [55, 56, 9003],
  'name': '',
  'rarity': 2,
  'tier': 2},
 {'character_id': 'TFT7_Varus',
  'itemNames': ['TFT_Item_MadredsBloodrazor', 'TFT_Item_Quicksilver'],
  'items': [12, 69],